In [7]:
import mne
import numpy as np
import scipy.io
import os
import sys

In [ ]:
# --- Set up Project Root ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
# Add 'src' directory to Python path to import config
sys.path.append(os.path.join(PROJECT_ROOT, 'src'))

In [ ]:
# --- Import configurations from config.py ---
# We'll try to import config, but this notebook is for verification so it might still work if config is slightly off.
try:
    from eeg.config import (
        RAW_MATLAB_DATA_DIR, 
        VEOG_CHANNEL_NAME, ACCEL_CHANNEL_NAMES, STIM_CHANNEL_NAME,
        ALL_SUBJECTS
    )
except ImportError:
    print("Warning: Could not import config.py. Proceeding with hardcoded paths/names for verification.")
    # Fallback paths/names if config.py isn't perfectly set up yet
    RAW_MATLAB_DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'matlab_eeg')
    VEOG_CHANNEL_NAME = 'VEOG' # Default assumption
    ACCEL_CHANNEL_NAMES = ['ACCEL_X', 'ACCEL_Y', 'ACCEL_Z'] # Default assumption
    STIM_CHANNEL_NAME = 'STI 014' # Default assumption
    ALL_SUBJECTS = [804, 890] # Default assumption for testing

In [ ]:
# --- Parameters for the specific .mat file you want to check ---
# Pick one subject and session from your actual data
TEST_SUBJ_ID = ALL_SUBJECTS[0] # Use the first subject in your defined list
TEST_SESSION_NUM = 1 # Test session 1

In [ ]:
# --- Construct the full path to the .mat file ---
# Based on your confirmed naming convention: [subjid]_[session]_PD_REST.mat
# OR, if you need to check the _REST1.mat version for any reason:
# mat_fname_base = f"{TEST_SUBJ_ID}_{TEST_SESSION_NUM}_PD_REST1"
mat_fname_base = f"{TEST_SUBJ_ID}_{TEST_SESSION_NUM}_PD_REST" 
mat_fname_full = os.path.join(RAW_MATLAB_DATA_DIR, f"{mat_fname_base}.mat")

In [3]:
subj_id = 804
session_num = 1
mat_fname_base = f"{subj_id}_{session_num}_PD_REST" # Test with the base version first
# Or, if your files are named like 804_1_PD_REST1.mat:
# mat_fname_base = f"{subj_id}_{session_num}_PD_REST1" 

mat_fname_full = os.path.join(RAW_MATLAB_DATA_DIR, f"{mat_fname_base}.mat")

print(f"Attempting to load MATLAB file: {mat_fname_full}")

Attempting to load MATLAB file: /Users/patriciawatanabe/Projects/NTUT25_Software/data/matlab_file/804_1_PD_REST.mat


In [4]:
try:
    mat_data = scipy.io.loadmat(mat_fname_full, simplify_cells=True, mat_dtype=True, squeeze_me=True)
    print(f"Successfully loaded {mat_fname_full}")

    print("\n--- Top-level keys in .mat file ---")
    print(mat_data.keys()) # Expect 'EEG' and '__header__', etc.

    eeg_struct = mat_data.get('EEG', None)
    if eeg_struct is None:
        print("Warning: 'EEG' key not found at top level. Assuming mat_data itself is the EEG struct.")
        eeg_struct = mat_data
    else:
        print("Found 'EEG' key. Proceeding with eeg_struct = mat_data['EEG']")

    print("\n--- Keys within the EEG struct ---")
    if isinstance(eeg_struct, dict):
        print(eeg_struct.keys()) # Look for 'data', 'srate', 'chanlocs', 'event'
    else:
        print(f"EEG struct is type {type(eeg_struct)}, not a dict. Cannot print keys.")


    print("\n--- Inspecting EEG.chanlocs ---")
    # Try different ways to access and print chanlocs
    if 'chanlocs' in eeg_struct:
        chanlocs = eeg_struct['chanlocs']
        print(f"Type of chanlocs: {type(chanlocs)}")
        if isinstance(chanlocs, np.ndarray) and chanlocs.ndim == 1 and len(chanlocs) > 0:
            print(f"Chanlocs is a NumPy array of length {len(chanlocs)}. Inspecting first element:")
            print(chanlocs[0]) # This should be a dictionary/struct representing a single channel
            if 'labels' in chanlocs[0]:
                print(f"Labels of first channel: {chanlocs[0]['labels']}")

        elif isinstance(chanlocs, dict): # Single chanlocs struct
            print(f"Chanlocs is a dictionary/struct. Inspecting it:")
            print(chanlocs)
            if 'labels' in chanlocs:
                print(f"Labels directly in chanlocs: {chanlocs['labels']}")

        else:
            print("Chanlocs has an unexpected type or structure. Please inspect manually in MATLAB.")

        # If the labels are directly a list/array of strings:
        if 'labels' in chanlocs and isinstance(chanlocs['labels'], (list, np.ndarray)) and all(isinstance(x, str) for x in chanlocs['labels']):
             print(f"\nDirect labels array found (first 5): {chanlocs['labels'][:5]}")
        else:
            print("\nDid not find 'labels' directly as a list/array of strings.")


    # --- Test raw_data_array and sfreq (for completeness) ---
    print("\n--- Inspecting EEG.data and EEG.srate ---")
    if 'data' in eeg_struct:
        data = eeg_struct['data']
        print(f"EEG.data shape: {data.shape}")
    if 'srate' in eeg_struct:
        srate = eeg_struct['srate']
        print(f"EEG.srate: {srate}")


except FileNotFoundError:
    print(f"Error: MATLAB file not found at {mat_fname_full}. Please check path and name.")
except Exception as e:
    print(f"An error occurred during interactive loading: {e}")
    import traceback
    traceback.print_exc()

Successfully loaded /Users/patriciawatanabe/Projects/NTUT25_Software/data/matlab_file/804_1_PD_REST.mat

--- Top-level keys in .mat file ---
dict_keys(['__header__', '__version__', '__globals__', 'EEG'])
Found 'EEG' key. Proceeding with eeg_struct = mat_data['EEG']

--- Keys within the EEG struct ---
dict_keys(['setname', 'filename', 'filepath', 'subject', 'group', 'condition', 'session', 'comments', 'nbchan', 'trials', 'pnts', 'srate', 'xmin', 'xmax', 'times', 'data', 'icaact', 'icawinv', 'icasphere', 'icaweights', 'icachansind', 'chanlocs', 'urchanlocs', 'chaninfo', 'ref', 'event', 'urevent', 'eventdescription', 'epoch', 'epochdescription', 'reject', 'stats', 'specdata', 'specicaact', 'splinefile', 'icasplinefile', 'dipfit', 'history', 'saved', 'etc'])

--- Inspecting EEG.chanlocs ---
Type of chanlocs: <class 'list'>
Chanlocs has an unexpected type or structure. Please inspect manually in MATLAB.

Did not find 'labels' directly as a list/array of strings.

--- Inspecting EEG.data and